# Demo for AIFlow class


Here is a list of methods in the `AIFlow` class with one-line descriptions:

1. **`__init__(self, api_key, model='gpt-4', temperature=0, max_tokens=150)`**: Initializes the AIFlow instance with given API key, model, temperature, and max tokens.
2. **`set_temperature(self, temperature=0)`**: Sets the temperature for the model.
3. **`set_model(self, model='gpt-4')`**: Sets the model to use.
4. **`set_max_tokens(self, max_tokens=150)`**: Sets the maximum number of tokens for responses.
5. **`show_model_config(self)`**: Prints the current model configuration.
6. **`show_self_data(self)`**: Prints the current state of chat messages, context map, images map, and audio map.
7. **`pretty_print_messages(self)`**: Prints chat messages in a formatted manner.
8. **`pretty_print_messages_to_file(self, file_name='output.txt', html=True)`**: Writes formatted chat messages to a file and optionally returns a download link.
9. **`set_system_prompt(self, prompt='')`**: Sets a system prompt message at the beginning of the chat messages.
10. **`add_user_chat(self, prompt, label='latest')`**: Adds a user chat message, gets a response from the API, and updates the context map.
11. **`filter_messages(self, func)`**: Filters chat messages based on a provided function.
12. **`reduce_messages_to_text(self, func)`**: Reduces chat messages to a text string using a provided function.
13. **`completion(self, prompt, label='latest')`**: Generates a completion for a given prompt and updates the context map.
14. **`call_openai_chat_api(self, messages=[])`**: Calls the OpenAI API to get a response for given messages.
15. **`replace_tags_with_content(self, input_string='')`**: Replaces tags in a string with corresponding context map values.
16. **`latest_to_context(self, label='latest')`**: Copies the latest context to a specified label.
17. **`transform_context(self, label='latest', func=lambda x: x)`**: Transforms a specified context using a provided function.
18. **`set_context(self, content='', label='latest')`**: Sets a specified context to the given content.
19. **`delete_context(self, label='latest')`**: Deletes a specified context from the context map.
20. **`show_context(self, label='latest')`**: Prints the content of a specified context.
21. **`read_textfile_to_context(self, filename, label='latest_file')`**: Reads a text file and sets its content to a specified context.
22. **`return_latest_to_text(self)`**: Returns the latest context as a text string.
23. **`return_context_to_text(self, label='latest')`**: Returns a specified context as a text string.
24. **`return_reduce_messages_to_text(self, func)`**: Reduces chat messages to a text string using a provided function and returns it.
25. **`dump_context_to_files(self)`**: Dump the context in a file, one file per context.
26. **`dump_context_to_markdown()`**: Dump the context into a MD formatted file.
27. **`dump_context_to_docx(output_filename='')`**: Dump the context into a word file.
28. **`save_state(filename='')`**: Save the state of all variables into a json file.
29. **`load_state(filename='')`**: Load the state of all variables from a json file.


## Magic
There is magic in the class - everytime you do a text prompt, the last result is stored in the context map under `latest`. So you can use that one as context variable for further processing or reference in queries.

## The demo

In [3]:
from dotenv import load_dotenv
import os
load_dotenv()
API_KEY=os.getenv("API_KEY")
from aiflow import AIFlow


# Just an empty flow

In [4]:
flow1 = AIFlow(api_key=API_KEY,model="gpt-3.5-turbo",max_tokens=3000, temperature=0)
(
    flow1
    .show_model_config()
    .show_self_data()
    .pretty_print_messages()
    .pretty_print_messages_to_file(file_name="nothing.txt")
)

Model: gpt-3.5-turbo
Max Tokens: 3000
Temperature: 0
Chat Messages:
[]

Context Map:
{}

Images Map:
{}

Audio Map:
{}


# Chat demo

In [5]:
chatbot=AIFlow(api_key=API_KEY,model="gpt-3.5-turbo",max_tokens=3000, temperature=1)
chatbot.show_model_config().show_self_data()

Model: gpt-3.5-turbo
Max Tokens: 3000
Temperature: 1
Chat Messages:
[]

Context Map:
{}

Images Map:
{}

Audio Map:
{}


In [6]:
(    chatbot
        .show_model_config()
        .set_system_prompt("You are a helpful cooking assistant always eager to give a recipe for whatever the user says.")
        .add_user_chat("How is the weather here?")
        .show_self_data()
)

Model: gpt-3.5-turbo
Max Tokens: 3000
Temperature: 1
You are a helpful cooking assistant always eager to give a recipe for whatever the user says.

How is the weather here?
I'm just a cooking assistant and I can't provide real-time weather updates. But if you're looking for a recipe to warm you up on a cold day, I recommend trying a hearty vegetable soup or a cozy mug of hot chocolate! Just let me know if you'd like a recipe for either.

Chat Messages:
[
    {
        "role": "system",
        "content": "You are a helpful cooking assistant always eager to give a recipe for whatever the user says."
    },
    {
        "role": "user",
        "content": "How is the weather here?"
    },
    {
        "role": "assistant",
        "content": "I'm just a cooking assistant and I can't provide real-time weather updates. But if you're looking for a recipe to warm you up on a cold day, I recommend trying a hearty vegetable soup or a cozy mug of hot chocolate! Just let me know if you'd like a 

In [7]:
chatbot.add_user_chat("Ok. Give me a recipe inspired by weather")

Ok. Give me a recipe inspired by weather
How about a refreshing and colorful Rainbow Fruit Salad? It's perfect for sunny and bright weather! Here's a simple recipe for you:

Ingredients:
- Strawberries
- Oranges
- Pineapple
- Kiwi
- Blueberries
- Blackberries
- Honey or agave nectar
- Fresh mint leaves (optional)

Instructions:
1. Wash and chop all the fruits into bite-sized pieces.
2. In a large bowl, combine the fruits and gently toss them together.
3. Drizzle honey or agave nectar over the fruit salad and gently mix to combine.
4. Garnish with fresh mint leaves for an extra pop of freshness.
5. Serve the colorful Rainbow Fruit Salad chilled and enjoy as a healthy and vibrant snack!

I hope you enjoy this delicious and weather-inspired recipe! Let me know if you need any more cooking inspiration.



In [8]:
chatbot.latest_to_context("dish")


In [9]:
chatbot.show_self_data()

Chat Messages:
[
    {
        "role": "system",
        "content": "You are a helpful cooking assistant always eager to give a recipe for whatever the user says."
    },
    {
        "role": "user",
        "content": "How is the weather here?"
    },
    {
        "role": "assistant",
        "content": "I'm just a cooking assistant and I can't provide real-time weather updates. But if you're looking for a recipe to warm you up on a cold day, I recommend trying a hearty vegetable soup or a cozy mug of hot chocolate! Just let me know if you'd like a recipe for either."
    },
    {
        "role": "user",
        "content": "Ok. Give me a recipe inspired by weather"
    },
    {
        "role": "assistant",
        "content": "How about a refreshing and colorful Rainbow Fruit Salad? It's perfect for sunny and bright weather! Here's a simple recipe for you:\n\nIngredients:\n- Strawberries\n- Oranges\n- Pineapple\n- Kiwi\n- Blueberries\n- Blackberries\n- Honey or agave nectar\n- Fresh 

## Message manipulations

In [10]:
chatbot.reduce_messages_to_text(lambda x: (print(x), "Replacing latest with all sorts of nothing")[1]).show_self_data()

[{'role': 'system', 'content': 'You are a helpful cooking assistant always eager to give a recipe for whatever the user says.'}, {'role': 'user', 'content': 'How is the weather here?'}, {'role': 'assistant', 'content': "I'm just a cooking assistant and I can't provide real-time weather updates. But if you're looking for a recipe to warm you up on a cold day, I recommend trying a hearty vegetable soup or a cozy mug of hot chocolate! Just let me know if you'd like a recipe for either."}, {'role': 'user', 'content': 'Ok. Give me a recipe inspired by weather'}, {'role': 'assistant', 'content': "How about a refreshing and colorful Rainbow Fruit Salad? It's perfect for sunny and bright weather! Here's a simple recipe for you:\n\nIngredients:\n- Strawberries\n- Oranges\n- Pineapple\n- Kiwi\n- Blueberries\n- Blackberries\n- Honey or agave nectar\n- Fresh mint leaves (optional)\n\nInstructions:\n1. Wash and chop all the fruits into bite-sized pieces.\n2. In a large bowl, combine the fruits and 

In [11]:
chatbot.filter_messages(lambda items: [{**item, "content": "You are an artist very good in poetry"} if item.get("role") == "system" else item for item in items])

In [12]:
chatbot.show_self_data()

Chat Messages:
[
    {
        "role": "system",
        "content": "You are an artist very good in poetry"
    },
    {
        "role": "user",
        "content": "How is the weather here?"
    },
    {
        "role": "assistant",
        "content": "I'm just a cooking assistant and I can't provide real-time weather updates. But if you're looking for a recipe to warm you up on a cold day, I recommend trying a hearty vegetable soup or a cozy mug of hot chocolate! Just let me know if you'd like a recipe for either."
    },
    {
        "role": "user",
        "content": "Ok. Give me a recipe inspired by weather"
    },
    {
        "role": "assistant",
        "content": "How about a refreshing and colorful Rainbow Fruit Salad? It's perfect for sunny and bright weather! Here's a simple recipe for you:\n\nIngredients:\n- Strawberries\n- Oranges\n- Pineapple\n- Kiwi\n- Blueberries\n- Blackberries\n- Honey or agave nectar\n- Fresh mint leaves (optional)\n\nInstructions:\n1. Wash and cho

## Just completions - no chat

In [13]:
chatbot.completion("Give me 10 random names")

Give me 10 random names

1. Olivia
2. Nathan
3. Sophia
4. Adrian
5. Isabella
6. Lucas
7. Ava
8. Elijah
9. Mia
10. Levi



In [14]:
chatbot.show_self_data()

Chat Messages:
[
    {
        "role": "system",
        "content": "You are an artist very good in poetry"
    },
    {
        "role": "user",
        "content": "How is the weather here?"
    },
    {
        "role": "assistant",
        "content": "I'm just a cooking assistant and I can't provide real-time weather updates. But if you're looking for a recipe to warm you up on a cold day, I recommend trying a hearty vegetable soup or a cozy mug of hot chocolate! Just let me know if you'd like a recipe for either."
    },
    {
        "role": "user",
        "content": "Ok. Give me a recipe inspired by weather"
    },
    {
        "role": "assistant",
        "content": "How about a refreshing and colorful Rainbow Fruit Salad? It's perfect for sunny and bright weather! Here's a simple recipe for you:\n\nIngredients:\n- Strawberries\n- Oranges\n- Pineapple\n- Kiwi\n- Blueberries\n- Blackberries\n- Honey or agave nectar\n- Fresh mint leaves (optional)\n\nInstructions:\n1. Wash and cho

In [15]:
chatbot.completion("Give the following names in reverse order: [latest].")

Give the following names in reverse order: [latest].

10. Levi
9. Mia
8. Elijah
7. Ava
6. Lucas
5. Isabella
4. Adrian
3. Sophia
2. Nathan
1. Olivia



## Saving and loading state
To avoid having the rerun all queries when you restart the kernel etc..

In [16]:
chatbot.save_state(filename="mystate.json")


In [17]:
chatbot.load_state(filename="mystate.json").show_self_data()

Chat Messages:
[
    {
        "role": "system",
        "content": "You are an artist very good in poetry"
    },
    {
        "role": "user",
        "content": "How is the weather here?"
    },
    {
        "role": "assistant",
        "content": "I'm just a cooking assistant and I can't provide real-time weather updates. But if you're looking for a recipe to warm you up on a cold day, I recommend trying a hearty vegetable soup or a cozy mug of hot chocolate! Just let me know if you'd like a recipe for either."
    },
    {
        "role": "user",
        "content": "Ok. Give me a recipe inspired by weather"
    },
    {
        "role": "assistant",
        "content": "How about a refreshing and colorful Rainbow Fruit Salad? It's perfect for sunny and bright weather! Here's a simple recipe for you:\n\nIngredients:\n- Strawberries\n- Oranges\n- Pineapple\n- Kiwi\n- Blueberries\n- Blackberries\n- Honey or agave nectar\n- Fresh mint leaves (optional)\n\nInstructions:\n1. Wash and cho

## Some context manipulations

In [18]:
chatbot.show_self_data()

Chat Messages:
[
    {
        "role": "system",
        "content": "You are an artist very good in poetry"
    },
    {
        "role": "user",
        "content": "How is the weather here?"
    },
    {
        "role": "assistant",
        "content": "I'm just a cooking assistant and I can't provide real-time weather updates. But if you're looking for a recipe to warm you up on a cold day, I recommend trying a hearty vegetable soup or a cozy mug of hot chocolate! Just let me know if you'd like a recipe for either."
    },
    {
        "role": "user",
        "content": "Ok. Give me a recipe inspired by weather"
    },
    {
        "role": "assistant",
        "content": "How about a refreshing and colorful Rainbow Fruit Salad? It's perfect for sunny and bright weather! Here's a simple recipe for you:\n\nIngredients:\n- Strawberries\n- Oranges\n- Pineapple\n- Kiwi\n- Blueberries\n- Blackberries\n- Honey or agave nectar\n- Fresh mint leaves (optional)\n\nInstructions:\n1. Wash and cho

In [19]:
chatbot.set_context(content="Nothing", label="stuff").show_self_data().delete_context(label="stuff").show_self_data()

Chat Messages:
[
    {
        "role": "system",
        "content": "You are an artist very good in poetry"
    },
    {
        "role": "user",
        "content": "How is the weather here?"
    },
    {
        "role": "assistant",
        "content": "I'm just a cooking assistant and I can't provide real-time weather updates. But if you're looking for a recipe to warm you up on a cold day, I recommend trying a hearty vegetable soup or a cozy mug of hot chocolate! Just let me know if you'd like a recipe for either."
    },
    {
        "role": "user",
        "content": "Ok. Give me a recipe inspired by weather"
    },
    {
        "role": "assistant",
        "content": "How about a refreshing and colorful Rainbow Fruit Salad? It's perfect for sunny and bright weather! Here's a simple recipe for you:\n\nIngredients:\n- Strawberries\n- Oranges\n- Pineapple\n- Kiwi\n- Blueberries\n- Blackberries\n- Honey or agave nectar\n- Fresh mint leaves (optional)\n\nInstructions:\n1. Wash and cho

In [20]:
chatbot.dump_context_to_files().dump_context_to_markdown().dump_context_to_docx(output_filename="output.docx")

## Data loading

In [21]:
chatbot.read_textfile_to_context(filename="output.txt").show_self_data()

Chat Messages:
[
    {
        "role": "system",
        "content": "You are an artist very good in poetry"
    },
    {
        "role": "user",
        "content": "How is the weather here?"
    },
    {
        "role": "assistant",
        "content": "I'm just a cooking assistant and I can't provide real-time weather updates. But if you're looking for a recipe to warm you up on a cold day, I recommend trying a hearty vegetable soup or a cozy mug of hot chocolate! Just let me know if you'd like a recipe for either."
    },
    {
        "role": "user",
        "content": "Ok. Give me a recipe inspired by weather"
    },
    {
        "role": "assistant",
        "content": "How about a refreshing and colorful Rainbow Fruit Salad? It's perfect for sunny and bright weather! Here's a simple recipe for you:\n\nIngredients:\n- Strawberries\n- Oranges\n- Pineapple\n- Kiwi\n- Blueberries\n- Blackberries\n- Honey or agave nectar\n- Fresh mint leaves (optional)\n\nInstructions:\n1. Wash and cho

# Combining AIFlows

In [22]:
otherbot=AIFlow(api_key=API_KEY,model="gpt-3.5-turbo",max_tokens=3000, temperature=1)
otherbot.completion("Let's say you are a random person hungry for food. What would you like to eat?")

Let's say you are a random person hungry for food. What would you like to eat?

As a language model AI, I don't have the ability to eat, but some popular food options for people who are hungry include pizza, burgers, pasta, tacos, sushi, or a hearty salad. Ultimately, the choice of what to eat depends on personal preferences and cravings.



In [23]:
otherbot.return_latest_to_text()

"As a language model AI, I don't have the ability to eat, but some popular food options for people who are hungry include pizza, burgers, pasta, tacos, sushi, or a hearty salad. Ultimately, the choice of what to eat depends on personal preferences and cravings."

In [24]:
sillybot=AIFlow(api_key=API_KEY,model="gpt-3.5-turbo",max_tokens=3000, temperature=1)
sillybot.set_context(content=otherbot.return_latest_to_text(), label="food_choice").show_self_data()
sillybot.completion(prompt="Give a silly comment on the following choice of food: [food_choice]").show_self_data()

Chat Messages:
[]

Context Map:
{
    "food_choice": "As a language model AI, I don't have the ability to eat, but some popular food options for people who are hungry include pizza, burgers, pasta, tacos, sushi, or a hearty salad. Ultimately, the choice of what to eat depends on personal preferences and cravings."
}

Images Map:
{}

Audio Map:
{}
Give a silly comment on the following choice of food: [food_choice]

I don't have taste buds, but I'm pretty sure all food tastes better when eaten in pajamas.

Chat Messages:
[]

Context Map:
{
    "food_choice": "As a language model AI, I don't have the ability to eat, but some popular food options for people who are hungry include pizza, burgers, pasta, tacos, sushi, or a hearty salad. Ultimately, the choice of what to eat depends on personal preferences and cravings.",
    "latest_prompt": "Give a silly comment on the following choice of food: As a language model AI, I don't have the ability to eat, but some popular food options for people

# Using vectordatabase Chroma

In [25]:
# https://docs.trychroma.com/guides
import chromadb
chroma_client = chromadb.Client() # chromadb.PersistentClient(path="/path/to/save/to")
collection = chroma_client.create_collection(name="my_collection")

In [26]:
collection.add(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2"]
)

In [27]:
results = collection.query(
    query_texts=["This is a query document about hawaii"], # Chroma will embed this for you
    n_results=1 # how many results to return
)
print(results)

{'ids': [['id1']], 'distances': [[1.0404009819030762]], 'metadatas': [[None]], 'embeddings': None, 'documents': [['This is a document about pineapple']], 'uris': None, 'data': None}


In [28]:
collection.peek() # returns a list of the first 10 items in the collection
collection.count() # returns the number of items in the collection

2

In [29]:
from aiflow import chroma_query_result_to_text
print(chroma_query_result_to_text(results))

This is a document about pineapple


In [30]:
sillybot.set_context(content=chroma_query_result_to_text(results), label="found_doc")
sillybot.completion("Say something really silly about this random text: [found_doc]").show_self_data()

Say something really silly about this random text: [found_doc]

Did you know that pineapples are actually tiny alien spaceships in disguise? Watch out for flying fruit in your grocery store!

Chat Messages:
[]

Context Map:
{
    "food_choice": "As a language model AI, I don't have the ability to eat, but some popular food options for people who are hungry include pizza, burgers, pasta, tacos, sushi, or a hearty salad. Ultimately, the choice of what to eat depends on personal preferences and cravings.",
    "latest_prompt": "Say something really silly about this random text: This is a document about pineapple",
    "latest": "Did you know that pineapples are actually tiny alien spaceships in disguise? Watch out for flying fruit in your grocery store!",
    "found_doc": "This is a document about pineapple"
}

Images Map:
{}

Audio Map:
{}
